In [1]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\grego\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\grego\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\grego\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
import numpy as np 
import pandas as pd 
from tqdm import tqdm 

import sqlite3
import pickle
import joblib

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin

In [3]:
database_filepath, model_filepath = "../data/DisasterResponse.db", "../models/classifier.pkl"

In [4]:
class GenreOneHotEncoding(BaseEstimator, TransformerMixin):

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        return pd.get_dummies(X)

In [5]:
def load_data(database_filepath):
    """
    Loads the database from a SQLite file into the script and split it in X, y and category names

    Parameters:
    database_filename (string): name of the database

    Returns:
    X (numpy array): Array containing the X values (messages)
    y (numpy array): Array containing the y values (one hot enconding of categories)
    category_names (list): List containing the category names in the database
    """
    conn = sqlite3.connect(database_filepath)

    # get a cursor
    cur = conn.cursor()

    # create the test table including project_id as a primary key
    df = pd.read_sql("SELECT * FROM '{}'".format(database_filepath), con=conn)

    conn.commit()
    conn.close()

    # genre_one_hot = pd.get_dummies(df["genre"])

    # X = pd.concat([df["message"], genre_one_hot], axis=1).values
    X = df[["message"]].values
    y = df.drop(["message", "genre"], axis=1).astype(int).values

    category_names = df.drop(["message", "genre"], axis=1).columns

    return X, y, category_names


def tokenize(text):
    """
    Tokenize the text to further process in the ML algorithm

    Parameters:
    text (string): the text from each message

    Returns:
    clean_tokens (list): The list of tokens processed
    """

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


def build_model():
    """
    Using grid search, builds the model to classify the messages

    Returns:
    model (): The trained model over the data
    """

    text_pipeline = Pipeline([
                              ('vect', CountVectorizer(tokenizer=tokenize)),
                              ('tfidf', TfidfTransformer())
                            ])

    genre_pipeline = Pipeline([
                               ('genre', GenreOneHotEncoding())
                             ])

    preprocessor = ColumnTransformer(transformers=[('text_pipeline', text_pipeline, 0),
                                                   ('num', genre_pipeline, 1)
                                                  ])

    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('clf', RandomForestClassifier())
    ])

    parameters = {
        'preprocessor__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'preprocessor__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'preprocessor__text_pipeline__vect__max_features': (None, 5000, 10000),
        'preprocessor__text_pipeline__tfidf__use_idf': (True, False),
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4]
    }

    cv = GridSearchCV(pipeline, param_grid=parameters)

    return cv


def evaluate_model(model, X_test, Y_test, category_names):
    """
    Evaluates the trained model

    Parameters:
    model (sklearn model): the trained model
    X_test (numpy array): An array with X values to test the model
    Y_test (numpy array): An array with Y values to test the model
    category_names (list): Names of the categories

    """
    Y_pred = model.predict(X_test)

    for i, category in enumerate(category_names):
        precision = precision_score(Y_test[i], Y_pred[i])
        recall = recall_score(Y_test[i], Y_pred[i])
        f1 = f1_score(Y_test[i], Y_pred[i])

        print("", category)
        print("Category: {} | F1-Score: {:.2f} % | Precision: {:.2f} % | Recall: {:.2f} %".format(category, 
                                                                                                  f1*100, 
                                                                                                  precision*100, 
                                                                                                  recall*100))
        # break
        # print("Overall Accuracy:", accuracy)

    # confusion_mat = confusion_matrix(Y_test, Y_pred, labels=category_names)
    # accuracy = (y_pred == y_test).mean()

    # print("Labels:", category_names)
    # print("Confusion Matrix:\n", confusion_mat)
    # print("Overall Accuracy:", accuracy)


def save_model(model, model_filepath):
    """
    Saves the trained model into a pickle file

    Parameters: 
    model (sklearn model): The trained model
    model_filepath (string): The path to save the model
    """
    joblib.dump(model, model_filepath)

In [6]:
print('Loading data...\n    DATABASE: {}'.format(database_filepath))
X, Y, category_names = load_data(database_filepath)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

# print('Building model...')
# model = build_model()

# print('Training model...')
# model.fit(X_train, Y_train)

# # print('Evaluating model...')
# evaluate_model(model, X_test, Y_test, category_names)

# # print('Saving model...\n    MODEL: {}'.format(model_filepath))
# save_model(model, model_filepath)

# print('Trained model saved!')

Loading data...
    DATABASE: ../data/DisasterResponse.db


In [7]:
pd.DataFrame(Y_train).describe()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
count,20998.000000,20998.000000,20998.000000,20998.000000,20998.000000,20998.000000,20998.000000,20998.000000,20998.000000,20998.0,...,20998.000000,20998.000000,20998.000000,20998.000000,20998.000000,20998.000000,20998.000000,20998.000000,20998.000000,20998.000000
mean,0.776741,0.170492,0.004715,0.417325,0.079246,0.050052,0.027526,0.017954,0.033289,0.0,...,0.011144,0.043195,0.276312,0.080817,0.092342,0.011049,0.092723,0.020478,0.051767,0.193114
std,0.434573,0.376074,0.068503,0.493129,0.270128,0.218058,0.163615,0.132788,0.179394,0.0,...,0.104977,0.203300,0.447184,0.272561,0.289515,0.104533,0.290051,0.141632,0.221561,0.394751
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
import joblib

In [17]:
type(X)

numpy.ndarray

In [10]:
X[0]

array(['weather update - a cold front from cuba that could pass over haiti'],
      dtype=object)

In [9]:
Y[0]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [11]:
category_names

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')